## Writing ome.zarr data

Write an example image, represented in memory by a numpy array, to an ome.zarr file on disc.

In [ ]:
# the ome_zarr imports we require
from czitools import pylibczirw_metadata as czimd
from czitools import pylibczirw_tools as czird
from czitools import misc, napari_tools
import napari
from pathlib import Path
import ome_zarr.reader
import ome_zarr.scale
import ome_zarr.writer
from ome_zarr.io import parse_url
import shutil
import zarr
zarr.__version__

In [ ]:
defaultdir = Path.cwd().resolve().parents[1] / "data"
filepath = defaultdir / "CellDivision_T=3_Z=5_CH=2_X=240_Y=170.czi"
zarr_path = defaultdir / Path(filepath.name[:-4] + ".ome.zarr")

print(zarr_path)

# check if path exists
remove = True
if zarr_path.exists() and remove:
    shutil.rmtree(zarr_path, ignore_errors=False, onerror=None)

In [ ]:
# get the metadata at once as one big class
mdata = czimd.CziMetadata(filepath)
print("Number of Scenes: ", mdata.image.SizeS)
scene_id = 0

In [ ]:
array5d, mdata, dim_string6d,  = czird.read_5darray(filepath,
                                                    scene=scene_id,
                                                    output_order="TCZYX",
                                                    output_dask=False,
                                                    chunks_auto=False,
                                                    remove_Adim=True)

array5d.shape

In [ ]:
ngff_version = ome_zarr.format.CurrentFormat().version
print("Using ngff format version", ngff_version)

# write the image data
store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)
root.info
print(root.tree())

In [ ]:
# write the OME-ZARR file
ome_zarr.writer.write_image(image=array5d, group=root, axes="tczyx", storage_options=dict(chunks=array5d.shape))

# convince yourself that the data is there
for child in zarr_path.iterdir():
    print(child)

In [ ]:
loc = ome_zarr.io.parse_url(zarr_path, mode="r")  # open the file in read mode
# this will return a reader object, which enables access to the individual resolution levels 
zarr_reader = ome_zarr.reader.Reader(loc).zarr

In [ ]:
# TODO is there a way to list the available resolution arrays?
# the 'load' functionality returns the specified resolution data as a dask array
res0 = zarr_reader.load("0")

In [ ]:
# the dask array can be used for lazy computation, or converted to numpy via .compute()
# for more information on dask arrays check out https://docs.dask.org/en/latest/array.html
full_image_npy = res0.compute()
print(full_image_npy.shape)

In [ ]:
# data slices can be used to select parts of the image.
# these will also be returned as dask arrays
sub_image = res0[:, :, :, :128, :128]
sub_image_npy = sub_image.compute()
print(sub_image_npy.shape)

## Using napari with ome.zarr

Use the napari plugin installed with `napari-ome-zarr` to open ome.zarr files directly.

In [ ]:
# this is how we can open the file we just wrote in napari
viewer = napari.Viewer()
viewer.open(zarr_path)